In [1]:
!pip3 install turicreate

     |████████████████████████████████| 92.0 MB 5.0 kB/s 
     |████████████████████████████████| 3.5 MB 51.8 MB/s 
     |████████████████████████████████| 86.4 MB 30 kB/s 
     |████████████████████████████████| 3.6 MB 39.8 MB/s 
     |████████████████████████████████| 322 kB 57.6 MB/s 
     |████████████████████████████████| 18.3 MB 81 kB/s 
     |████████████████████████████████| 3.8 MB 52.6 MB/s 
     |████████████████████████████████| 449 kB 50.8 MB/s 
     |████████████████████████████████| 50 kB 9.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 55.1 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=2bd528583883f60f356b9f130e21bb3e2d975c8a4e17a0323284d42b609f65cc
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320860 sha256=8be0d09f57a

In [2]:
import pandas as pd
import turicreate as tc

In [3]:
# Read in data
# Student info table
raw_url = 'https://drive.google.com/file/d/1sQ4OZAzxUi5zX_xjDz-b4RAJ6imPuczq/view?usp=sharing'
url ='https://drive.google.com/uc?id=' + raw_url.split('/')[-2]

std_df = pd.read_csv(url, index_col = 0)

# Activities table
raw_url = 'https://drive.google.com/file/d/1Z_IW3O2ckA1-w0tWEZMS7Bmd5dBIglNO/view?usp=sharing'
url ='https://drive.google.com/uc?id=' + raw_url.split('/')[-2]

act_df = pd.read_csv(url, index_col = 0)

# Courseplan table
raw_url = 'https://drive.google.com/file/d/1NEFGcm4hG08UZ8S_W1kDzYXZ_rCu0USe/view?usp=sharing'
url ='https://drive.google.com/uc?id=' + raw_url.split('/')[-2]

courseplan_df = pd.read_csv(url, index_col = 0)

# Student activity table
raw_url = 'https://drive.google.com/file/d/1ai080vOvAM3a82AArVRA86QwwWb5I5rx/view?usp=sharing'
url ='https://drive.google.com/uc?id=' + raw_url.split('/')[-2]

std_act_df = pd.read_csv(url, index_col = 0)

# Student performance table
raw_url = 'https://drive.google.com/file/d/1dKR6tH0DcBXZXcSycUl38wfqlwRQzALY/view?usp=sharing'
url ='https://drive.google.com/uc?id=' + raw_url.split('/')[-2]

std_performance_df = pd.read_csv(url, index_col = 0)

In [4]:
print(std_act_df.shape)
std_act_df.head()

(7320, 13)


,Stamp,Activity_ID,User_ID,Hits,Misses,Max_Hits,GamePlayTime,Scores,Translanguage_Level_1,Translanguage_Level_2,Words,Course_ID,Usefulness
0,2021-09-27 03:19:31,easter-colors-game-2,0,8,7,8,279.483906,64.299357,1,3,"['orange', 'purple', 'purple', 'brown', 'black...",0_0,26.500116
1,2021-09-30 22:21:26,easter-colors-game-2,0,8,5,8,287.051023,71.992602,1,3,"['purple', 'white', 'black', 'white', 'blue', ...",0_0,21.884169
2,2021-10-03 00:54:19,pirate-all-puzzle-1,0,8,4,8,275.561071,76.455156,1,2,"['two', 'eight', 'two', 'two', 'four', 'seven'...",0_0,19.206637
3,2021-10-04 08:03:06,values-all-book-20,0,1,0,1,270.096018,84.479003,1,1,['stressed'],0_0,14.392328
4,2021-10-06 21:28:46,values-all-book-22,0,1,0,1,274.757235,84.414754,1,1,['sad'],0_0,14.430878


In [5]:
train_data = tc.SFrame(std_act_df)  # Data needs to be converted to SFrame format

In [6]:
# Used for quick visualisation and data exploration
train_data.show()

Materializing SFrame

## Train/Test Split

In [7]:
# 80/20 Train/Test Split
train, test = tc.recommender.util.random_split_by_user(train_data, user_id='User_ID', item_id='Activity_ID', item_test_proportion=0.2)

In [8]:
len(train), len(test)

(5840, 1480)

# Popularity recommender system

In [9]:
# training the model
popularity_model = tc.popularity_recommender.create(train, user_id='User_ID', item_id='Activity_ID', target='Usefulness')

Warning: Ignoring columns Stamp, Hits, Misses, Max_Hits, GamePlayTime, Scores, Translanguage_Level_1, Translanguage_Level_2, Words, Course_ID;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 5840 observations with 250 users and 171 items.

Data prepared in: 0.019829s

5840 observations to process; with 171 unique items.

In [10]:
# Recommend for a given set of users, since there are top 5 recommendation for 3 users, total rows will be 15
popularity_recomm = popularity_model.recommend(users=[0,1,2],k=5)
popularity_recomm.print_rows(num_rows=15)

+---------+-------------------------------+--------------------+------+
| User_ID |          Activity_ID          |       score        | rank |
+---------+-------------------------------+--------------------+------+
|    0    |       vegetables-poiki-4      | 68.80968056138033  |  1   |
|    0    |      colors-yasmin-game-3     | 45.00789653547849  |  2   |
|    0    |      country-none-book-2      | 40.48894536760133  |  3   |
|    0    |    sea-yasmin-flashcards-1    | 38.753641933459726 |  4   |
|    0    | instruments-yasmin-flashca... | 38.609709682766386 |  5   |
|    1    |       vegetables-poiki-4      | 68.80968056138033  |  1   |
|    1    |      colors-yasmin-game-3     | 45.00789653547849  |  2   |
|    1    |      country-none-book-2      | 40.48894536760133  |  3   |
|    1    |    sea-yasmin-flashcards-1    | 38.753641933459726 |  4   |
|    1    | instruments-yasmin-flashca... | 38.609709682766386 |  5   |
|    2    |       vegetables-poiki-4      | 68.80968056138033  |

In [11]:
popularity_model_performance = popularity_model.evaluate(test)


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |          0.0          |          0.0          |
|   2    |  0.004184100418410042 |  0.000843852185225555 |
|   3    |  0.002789400278940028 | 0.0008438521852255547 |
|   4    |  0.002092050209205021 | 0.0008438521852255545 |
|   5    | 0.0033472803347280337 | 0.0017271622735565634 |
|   6    |  0.002789400278940028 | 0.0017271622735565634 |
|   7    | 0.0023909145248057376 | 0.0017271622735565634 |
|   8    |  0.002615062761506276 | 0.0021075350388665673 |
|   9    | 0.0027894002789400274 |  0.002525945080707572 |
|   10   | 0.0025104602510460255 | 0.0025259450807075716 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 13.258825996501573

Per User RMSE (best)
+---------+-------------------+-------+
| Us

In [12]:
popularity_model_performance['rmse_overall']

13.258825996501573

In [13]:
user1_recomm_popularity_model = popularity_model.recommend(users=[0], k=5)
df_user1_recomm_popularity_model = user1_recomm_popularity_model.to_dataframe()
df_user1_recomm_popularity_model

,User_ID,Activity_ID,score,rank
0,0,vegetables-poiki-4,68.809681,1
1,0,colors-yasmin-game-3,45.007897,2
2,0,country-none-book-2,40.488945,3
3,0,sea-yasmin-flashcards-1,38.753642,4
4,0,instruments-yasmin-flashcards-1,38.609710,5


In [14]:
popularity_model.save("turicreate_trending_model")

In [17]:
#function which takes saved model file, user id for which recommendations need to done 
#and num of recommendations as parameter and returns the recommendations as output

def make_recommendations(model, user_id, num_recommendations):
  model_tc = tc.load_model(model)
  user_recomm = model_tc.recommend(users=[user_id], k=num_recommendations)
  df_user_recomm = user_recomm.to_dataframe()
  return df_user_recomm

In [18]:
make_recommendations("turicreate_trending_model", 0, 5)

,User_ID,Activity_ID,score,rank
0,0,vegetables-poiki-4,68.809681,1
1,0,colors-yasmin-game-3,45.007897,2
2,0,country-none-book-2,40.488945,3
3,0,sea-yasmin-flashcards-1,38.753642,4
4,0,instruments-yasmin-flashcards-1,38.609710,5
